In [1]:
%pylab
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from scipy import fftpack
import cv2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Limpieza de imagenes

In [2]:
# Cargar un imagen
image = io.imread('images/moonlanding.png')
f, ax = plt.subplots(figsize=(4,4))
ax.imshow(image)

In [3]:
M, N = image.shape
print( (M,N), image.dtype )

((474, 630), dtype('uint8'))


In [4]:
# Tomemos la transformada de Fourier

FFT = fftpack.fftn(image)

# Tomar la norma del coeficiente de Fourier
FFTA = np.abs(FFT)

# Correr las frecuencias bajas al cero
FFTA = fftpack.fftshift(FFTA)

In [5]:
f, ax = plt.subplots(figsize=(5,5))
ax.imshow(np.log(1+FFTA), cmap='viridis',
          extent=(-N // 2, N // 2, -M // 2, M //2))


ax.set_title('Spectrum')

Text(0.5,1,'Spectrum')

In [6]:
# Ajustar el centro del espectro a cero
K = 40
FFTA[M // 2 - K: M // 2 + K, N // 2 - K: N // 2 + K] = 0

In [7]:
# Encontrar los picos en el espacio de frecuencias 
picos = FFTA < np.percentile(FFTA, 98)

# Alinear picos con el espectro original

picos = fftpack.ifftshift(picos)

In [8]:
# Vamos a hacer una copia de la transformada de Fourier
FFTCopy = FFT.copy()

# Ajustar picos a cero

FFTCopy = FFTCopy * picos.astype(int)

In [9]:
f, ax = plt.subplots(figsize=(5,5))
ax.imshow(np.log10(1.+ np.abs(FFTCopy)), cmap='viridis')
ax.set_title('Spectrum despues de la supresion')

Text(0.5,1,'Spectrum despues de la supresion')

In [10]:
# Encontra la nueva imagen filtrada. 
image_filtrada = np.real( fftpack.ifft2(FFTCopy) )

In [11]:
f, ax = plt.subplots(figsize=(5,5))
ax.imshow(image_filtrada)
ax.set_title('Imagen_Filtrada')

Text(0.5,1,'Imagen_Filtrada')

# References

https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html

# Bordes.

Lugares de la imagen donde hay un cambio brusco de la intensidad.

$$ dI(x,y) = \sqrt{ \bigg(\frac{dI(x,y)}{dx}\bigg)^{2} + \bigg(\frac{dI(x,y)}{dx}\bigg)^{2} }  $$

In [12]:
# !pip install opencv-python

In [13]:
image = cv2.imread('images/car.png')
f, ax = plt.subplots(figsize=(4,4))
ax.imshow(image)

In [14]:
# Redimensionar a imagen cuadrada

Nx, Ny = 512,512
image = cv2.resize(image, (Nx,Ny))
cv2.imshow('ImagenOriginal',image)
# Pasar a escala de grises
image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

In [15]:
kernel1=np.array([-0.5,0,0.5])
kernel2=np.array([[-0.5],[0],[0.5]])

# Filtros de Sobel
Hsx=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
Hsy=np.transpose(Hsx)

bordex = cv2.filter2D(image, -1, kernel1)
bordey = cv2.filter2D(image, -1, kernel2)


# Calcular la Magnitud

Mxy = np.sqrt(bordex**2 + bordey**2)

# Normalizacion a 1
Mxy = Mxy/np.max(Mxy)

# Aplicar el filtro, retorna color blanco si se cumple la condicion
mask = np.where(Mxy>0.1,255,0)
mask = np.uint8(mask)
cv2.imshow('Border',mask)

In [ ]:
video = cv2.VideoCapture(0)

while True:
    # Lectura de imagenes
    ret, frame1 = video.read()
    frame = cv2.cvtColor(frame1,cv2.COLOR_RGB2GRAY)
    
    # Conversion de entero a flotante
    frame_float = frame.astype(float)
    
    # Kernel De Sobel
    Hsx=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    Hsy=np.transpose(Hsx)
    
    bordex = cv2.filter2D(frame_float,-1,Hsx)
    bordey = cv2.filter2D(frame_float,-1,Hsy)
    
    # Calcular la magnitud
    Mxy = np.sqrt(bordex**2+bordey**2)
    
    # Normalizacion a 1
    Mxy = Mxy/np.max(Mxy)
    
    # Segmentacion retorna blanco si es verdadero
    mask = np.where(Mxy>0.2,255,0)
    mask = np.uint8(mask)
    
    cv2.imshow('Border',mask)
    
    k=cv2.waitKey(1)&0xFF
    if(k==ord('p')):
        print('Exit')
        break

video.release()
cv2.destroyAllWindows()
    